# Chunking Strategien - Einfache Erklärung

Um zu zeigen wie chunking funktioniert nehmen wir uns folgenden Beispieltext:


In [ ]:
beispiel_text = """
Künstliche Intelligenz (KI) ist ein Teilgebiet der Informatik. Sie befasst sich mit der Automatisierung intelligenten Verhaltens. Warum ist KI so wichtig? Die Antwort liegt in ihren vielfältigen Anwendungsmöglichkeiten!
Machine Learning ist eine wichtige Methode der KI. Dabei lernen Computer aus Daten, ohne explizit programmiert zu werden. Es gibt verschiedene Arten: überwachtes Lernen, unüberwachtes Lernen und verstärkendes Lernen. Welche Methode ist die beste? Das hängt vom Anwendungsfall ab!
Deep Learning ist eine spezielle Form des Machine Learning. Es verwendet neuronale Netze mit vielen Schichten. Diese Netze können komplexe Muster in großen Datenmengen erkennen. Besonders erfolgreich ist Deep Learning in der Bildverarbeitung und Spracherkennung!

Large Language Models (LLMs) sind eine neuere Entwicklung. Sie basieren auf der Transformer-Architektur. Modelle wie GPT können Text generieren, übersetzen und Fragen beantworten. Sie werden mit riesigen Textmengen trainiert. Sind LLMs die Zukunft der KI? Viele Experten glauben das!
Die Anwendungen von KI sind vielfältig. In der Medizin hilft KI bei der Diagnose von Krankheiten. In der Industrie optimiert sie Produktionsprozesse. Im Alltag nutzen wir KI in Sprachassistenten und Empfehlungssystemen. Wo wird KI als nächstes eingesetzt? Die Möglichkeiten scheinen grenzenlos!
"""

## Character-based Chunking

Character-based Chunking ist die **einfachste Chunking-Methode**. Der Text wird einfach nach einer festen Anzahl von Zeichen aufgeteilt, unabhängig von Wort- oder Satzgrenzen.

**So funktioniert es:**

1. Der Text wird in Stücke mit einer **festen Zeichenlänge** (z.B. 200 Zeichen) aufgeteilt
2. Optional kann ein **Overlap** (Überlappung) definiert werden, sodass die Enden der Chunks sich überschneiden
3. Die Aufteilung erfolgt **ohne Rücksicht** auf Wörter, Sätze oder semantische Grenzen

**Vorteile:**
- Sehr **einfach** zu implementieren
- **Vorhersagbare** Chunk-Größen
- **Schnell** in der Ausführung

**Nachteile:**
- **Keine semantische Struktur**: Sätze und Wörter werden mitten durchgeschnitten
- **Schwer lesbar**: Chunks können mitten im Wort enden
- **Schlechte Retrieval-Qualität**: Zusammenhängende Informationen werden oft getrennt

Schauen wir uns an, wie das in der Praxis aussieht:

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

def test_character_chunking(chunk_size=200, chunk_overlap=20, separator=""):
    """
    Demonstriert Character-based Chunking
    
    Parameters:
    - chunk_size: Anzahl der Zeichen pro Chunk
    - chunk_overlap: Anzahl der überlappenden Zeichen zwischen Chunks
    - separator: Trennzeichen (leer = strikte Zeichentrennung)
    """
    character_splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separator=separator,
        length_function=len,
    )
    
    chunks = character_splitter.split_text(beispiel_text)
    
    print(f"Anzahl der Chunks: {len(chunks)}")
    print(f"Chunk-Größe: {chunk_size} Zeichen, Overlap: {chunk_overlap} Zeichen\n")
    print("=" * 80)
    
    for i, chunk in enumerate(chunks, 1):
        print(f"\nChunk {i} ({len(chunk)} Zeichen):")
        print("-" * 80)
        print(chunk)
        print("-" * 80)
        
        # Zeige die Überlappung mit dem nächsten Chunk
        if i < len(chunks) and chunk_overlap > 0:
            overlap_text = chunk[-chunk_overlap:]
            next_overlap = chunks[i][:chunk_overlap]
            if overlap_text in chunks[i]:
                print(f"Überlappung mit nächstem Chunk: '{overlap_text}'")

# Beispiel 1: Ohne Overlap - Sätze werden hart abgeschnitten!
print("=== BEISPIEL 1: Chunk-Größe 200, KEIN Overlap ===\n")
test_character_chunking(chunk_size=200, chunk_overlap=0)

### Experimentiere mit verschiedenen Parametern

Passe im Folgenden die `chunk_size` und `chunk_overlap` Werte an, um zu sehen wie sich die Chunks verändern.

**Interessante Szenarien zum Ausprobieren:**

1. **Sehr kleine Chunks** (chunk_size=100, chunk_overlap=0)
   - Zeigt extreme Fragmentierung
   - Sätze werden stark zerschnitten

2. **Kleine Chunks mit Overlap** (chunk_size=100, chunk_overlap=30)
   - Overlap von 30% hilft, zerbrochene Wörter zu "reparieren"
   - Aber erhöht Redundanz

3. **Mittlere Chunks ohne Overlap** (chunk_size=300, chunk_overlap=0)
   - Weniger Fragmentierung
   - Aber immer noch harte Schnitte

4. **Mittlere Chunks mit moderatem Overlap** (chunk_size=300, chunk_overlap=50)
   - Balance zwischen Größe und Kontinuität

5. **Großer Overlap** (chunk_size=200, chunk_overlap=150)
   - 75% Überlappung!
   - Sehr hohe Redundanz, aber maximale Kontinuität

Probiere diese Werte aus und achte darauf:
- Wo werden Wörter zerschnitten?
- Wie viel Redundanz entsteht durch Overlap?
- Welche Konfiguration ist am besten lesbar?

In [ ]:
# Beispiel 2: Experimentiere mit verschiedenen Werten!
# Tipp: Probiere die oben genannten Szenarien aus

# Szenario 4: Mittlere Chunks mit moderatem Overlap
test_character_chunking(chunk_size=300, chunk_overlap=50)

### Das Problem: Sätze werden mitten durchgeschnitten

Wie du in den Beispielen oben siehst, wird der Text **strikt nach Zeichenzahl** aufgeteilt:

- **Chunk 1** endet möglicherweise mitten im Satz: "...Künstliche Intelligenz (KI) ist ein Teilgebiet der Informatik. Sie befasst sich mit der Automatisierung intelligenten Verhaltens. Warum ist KI so wichtig? Die Antwort liegt in ihren vielfälti"
- **Chunk 2** beginnt dann mit: "gen Anwendungsmöglichkeiten!..."

Das Wort "vielfältigen" wird in **"vielfälti"** und **"gen"** zerschnitten!

**Overlap hilft nur teilweise:**
- Mit Overlap von 50 Zeichen werden die letzten 50 Zeichen des vorherigen Chunks im nächsten Chunk wiederholt
- Das reduziert das Problem, aber verhindert es nicht vollständig
- Außerdem erhöht es die Gesamtgröße der Daten durch Duplikation

**Fazit:** Character-based Chunking ist einfach, aber für die meisten RAG-Anwendungen **nicht empfohlen**. Besser geeignet sind semantisch bewusste Methoden wie Recursive Chunking (siehe unten).

## Recursive Chunking

Recursive Chunking ist eine hierarchische Methode, um Text in kleinere Teile (Chunks) zu zerlegen. Der Name "recursive" (rekursiv) kommt daher, dass der Algorithmus verschiedene Trennzeichen nacheinander ausprobiert.

**So funktioniert es:**

1. Der Algorithmus versucht zuerst, den Text mit dem **ersten Separator** (z.B. `\n\n` - doppelte Zeilenumbrüche) zu trennen
2. Wenn die resultierenden Teile noch zu groß sind, versucht er den **nächsten Separator** (z.B. `\n` - einfache Zeilenumbrüche)
3. Dieser Prozess wiederholt sich durch die Liste der Separatoren (z.B. `.`, ` `, ``) bis die Chunk-Größe erreicht ist

**Warum ist das nützlich?**

- Es erhält **natürliche Textgrenzen**: Absätze bleiben zusammen, wenn möglich
- Es ist **flexibel**: Falls nötig, wird an kleineren Einheiten getrennt (Sätze, Wörter, Zeichen)
- Es ist **semantisch sinnvoll**: Zusammenhängende Informationen bleiben eher beisammen

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def test_chunking(chunk_size=500,chunk_overlap=50,separators=["\n\n", "\n", ".", " "]):
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=separators,
    )

    chunks = recursive_splitter.split_text(beispiel_text)

    print(f"Anzahl der Chunks: {len(chunks)}\n")
    print("="*80)

    for i, chunk in enumerate(chunks, 1):
        print(f"\nChunk {i} ({len(chunk)} Zeichen):")
        print("-" * 80)
        print(chunk)
        print("-" * 80)

test_chunking()

### Aufgabe 1: Separatoren verstehen
Einer der Chunks ist signifikant größer als die anderen.

**Wie funktionieren Separatoren?**

Der RecursiveCharacterTextSplitter arbeitet die Separatoren-Liste hierarchisch ab:
1. Zuerst versucht er mit `\n\n` (Absatzgrenzen) zu trennen
2. Wenn Chunks noch zu groß sind, nutzt er `\n` (Zeilenumbrüche)
3. Danach kommt `.` (Satzenden mit Punkt)
4. Als letztes würde er bei einzelnen Zeichen trennen

**Problem:** Chunk 1 ist mit 499 Zeichen fast doppelt so groß wie die anderen!

**Ursache:** Im Text enden viele Sätze mit `!` oder `?`, aber diese Zeichen fehlen in der Separatoren-Liste. Daher werden zusammenhängende Textblöcke nicht optimal getrennt.

**Lösung:** Füge die fehlenden Satzzeichen `!` und `?` als Separatoren hinzu:

#### Experiment 1: Kleine Chunks (chunk_size=200)

Was passiert, wenn wir sehr kleine Chunks verwenden?

In [ ]:
# Teste mit kleiner chunk_size
test_chunking(chunk_size=200, chunk_overlap=50)

#### Experiment 2: Große Chunks (chunk_size=1000)

Wie sehen die Chunks aus, wenn wir eine größere Chunk-Größe verwenden?

In [ ]:
# Teste mit großer chunk_size
test_chunking(chunk_size=1000, chunk_overlap=50)

## Chunking-Strategie evaluieren

Nachdem du verschiedene Chunking-Parameter getestet hast, möchtest du herausfinden, welche Konfiguration die beste Retrieval-Qualität liefert.

### Schritt-für-Schritt Anleitung zur Evaluation

#### 1. Chunking-Parameter in `.env` anpassen und Evaluierung starten

Öffne die `.env` Datei und passe die folgenden Variablen an:

```bash
CHUNK_SIZE=5000          # Gewünschte Chunk-Größe in Zeichen
CHUNK_OVERLAP=500        # Überlappung zwischen Chunks
```

- Backend neu starten, damit die neuen Chunking-Parameter wirksam werden
- Evaluation ausführen


#### 4. Ergebnisse in Langfuse analysieren

1. Öffne das Langfuse Dashboard
2. Suche nach den neuesten Evaluation-Runs
3. Vergleiche die Metriken:
   - **Context Precision**: Wie relevant sind die abgerufenen Chunks?
   - **Context Recall**: Werden alle relevanten Informationen gefunden?
   - **Answer Relevancy**: Wie gut beantwortet das System die Frage?

#### 5. Verschiedene Konfigurationen vergleichen

Wiederhole Schritte 1-4 mit verschiedenen `CHUNK_SIZE` und `CHUNK_OVERLAP` Werten:
- Kleine Chunks (z.B. 1000): Präziser, aber möglicherweise fehlender Kontext
- Große Chunks (z.B. 10000): Mehr Kontext, aber weniger präzise Retrieval
- Overlap: Hilft bei Chunks, die an wichtigen Stellen getrennt wurden

**Tipp:** Notiere dir die Scores für jede Konfiguration, um die optimalen Parameter zu finden!